<a href="https://colab.research.google.com/github/ShreyShukla14001/galytix/blob/main/galytix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import gensim
import numpy
from gensim.models import KeyedVectors
download_path='/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz'
wv=KeyedVectors.load_word2vec_format(download_path,binary=True,limit=1000000)

In [4]:
wv.save_word2vec_format('vectors.csv')

In [6]:
import pandas as pd
df=pd.read_csv('/content/phrases (1).csv',encoding='unicode_escape')

In [8]:
def assign_embeddings(df):
    phrase_embeddings=[]
    for index,rows in df.iterrows():
        phrase=rows['Phrases']
        words=phrase.split()
        word_embeddings=[]
        for word in words:
            if word in wv:
                word_embeddings.append(wv[word])
        if word_embeddings:
            phrase_embedding=sum(word_embeddings)/len(word_embeddings)
            phrase_embeddings.append(phrase_embedding)
        else:
            phrase_embeddings.append(None)
    return phrase_embeddings
assign_embeddings(df)

[array([ 0.10018921,  0.02929688,  0.09875488,  0.01054382,  0.01702881,
         0.06987762,  0.05764771, -0.08322144,  0.12287903, -0.07568359,
        -0.1315918 , -0.05178833,  0.11340332, -0.02423096, -0.24975586,
         0.12243652, -0.07659912,  0.1335907 , -0.00775146, -0.10144043,
        -0.07360077,  0.03451538, -0.21917725, -0.00453186, -0.06338501,
         0.07003784, -0.05845976,  0.18017578,  0.08743286,  0.00067139,
         0.07165527,  0.00500488, -0.08312988,  0.04141235,  0.05331421,
        -0.00314331, -0.0637207 , -0.0362854 ,  0.11993408,  0.07693481,
         0.11108398, -0.00527954,  0.05410767,  0.1239624 , -0.11419678,
        -0.05019569, -0.04246521,  0.06715584, -0.00283813,  0.01797485,
         0.10693359, -0.01367188,  0.00115967, -0.10668945, -0.00112915,
        -0.04327393, -0.16442871, -0.00959778,  0.18981934,  0.10974121,
         0.00172043,  0.15270996, -0.09210205, -0.07113647,  0.00798798,
        -0.02490234, -0.00115967, -0.01260376,  0.0

In [13]:
from scipy.spatial.distance import cosine
import numpy as np

def cosine_distance(df):
    phrase_embeddings=assign_embeddings(df)
    num_phrases = len(phrase_embeddings)
    l2_distances = np.zeros((num_phrases, num_phrases))
    cosine_distances = np.zeros((num_phrases, num_phrases))
    phrase_embeddings=assign_embeddings(df)
    num_phrases=len(phrase_embeddings)
    # Calculate distances for all pairs of phrases
    cosine_distances=np.zeros((num_phrases,num_phrases))
    for i in range(num_phrases):

        for j in range(i, num_phrases):
            # Calculate Cosine distance
            cosine_dist = cosine(phrase_embeddings[i], phrase_embeddings[j])
            cosine_distances[i][j] = cosine_dist
            cosine_distances[j][i] = cosine_dist

    return cosine_distances
cosine_distance(df)

array([[0.        , 0.48511559, 0.63066179, ..., 0.5808233 , 0.43262547,
        0.6501182 ],
       [0.48511559, 0.        , 0.68823931, ..., 0.53877673, 0.53459084,
        0.52022618],
       [0.63066179, 0.68823931, 0.        , ..., 0.58025837, 0.61000773,
        0.63217899],
       ...,
       [0.5808233 , 0.53877673, 0.58025837, ..., 0.        , 0.44645053,
        0.26985687],
       [0.43262547, 0.53459084, 0.61000773, ..., 0.44645053, 0.        ,
        0.40822583],
       [0.6501182 , 0.52022618, 0.63217899, ..., 0.26985687, 0.40822583,
        0.        ]])

In [16]:
def find_closest_match(input_string,df):
    phrase_embeddings=assign_embeddings(df)

    input_embeddings=[]
    input_words=input_string.split()

    for word in input_words:
        if word in wv:
            input_embeddings.append(wv[word])
    if not input_embeddings:
        return "No matching phrase"
    input_embedding=sum(input_embeddings)/len(input_embeddings)
    cosine_distances=[cosine(input_embedding,phrase_emb) for phrase_emb in phrase_embeddings]
    closest_match_index=np.argmin(cosine_distances)
    closest_match_phrase=df['Phrases'][closest_match_index]
    closest_match_distance=cosine_distances[closest_match_index]

    return closest_match_phrase,closest_match_distance
find_closest_match("which is the best airport",df)


('How big is the airlines industry in China?', 0.3643491864204407)